In [7]:
def display_html(df, cols=None, num_rows=0):
    """Display columns cols and num_rows rows of the data 
    frame df in HTML.
    """
    
    if num_rows != 0:
        df_to_display = df.head(num_rows)
    else:
        df_to_display = df
    
    df_html = df_to_display.to_html(columns=cols, index=False, na_rep='',
                              escape=False, render_links=True)
    display(HTML(df_html))

In [ ]:
import kagglehub

path = kagglehub.dataset_download("kaggle/meta-kaggle")
print("Path to dataset files:", path)

In [ ]:
# sournce: https://www.kaggle.com/code/sahidvelji/meet-the-grandmasters 
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

import time
import re
import cssutils
import json
import matplotlib.pyplot as plt

from IPython.display import display, HTML
        
KAGGLE_BASE_URL = 'https://kaggle.com/'
TWITTER_BASE_URL = 'https://twitter.com/'
GITHUB_BASE_URL = 'https://github.com/'
DataREPO = './data/1737/'
#TOP_N = 10

# modified
import os
def beep():
    os.system('afplay /System/Library/Sounds/Ping.aiff')

import pickle

In [ ]:
# Users
users = pd.read_csv(DataREPO+'Users.csv', encoding='utf-8')
user_ach = pd.read_csv(DataREPO+'UserAchievements.csv', encoding='utf-8')

beep() # 

print(f"There are {users.shape[0]} data in unique users.")
print(f"There are {users.shape[0]} data in user_ach, which means number of users which accomplish some achievements")


In [89]:
# PerformanceTier: 2(expert),3(master),4(grandmaster)
pftier_idx = 3
pftier = ['Novice','Contributer','Expert','Master','Grandmaster']
gms = (users[users['PerformanceTier'] == pftier_idx]
        .copy()
        .reset_index(drop=True)
        .drop(columns=['PerformanceTier'])
      )
print(f"There are currently unique {gms.shape[0]} {pftier[pftier_idx]}s.")

gms_ach = (user_ach[user_ach['Tier'] == pftier_idx]
            .copy()
            .reset_index(drop=True)
            .drop(columns=['Tier','TierAchievementDate','Points','CurrentRanking','HighestRanking','TotalGold','TotalSilver','TotalBronze'])
          )
print(f"There are currently {gms_ach.shape[0]} {pftier[pftier_idx]}s in total. Note that the number includes double achievement")



There are currently unique 2990 Masters.
There are currently 3349 Masters in total. Note that the number includes double achievement


In [ ]:
#print(gms.head(3))
uid = gms_ach.UserId.unique()
print(len(uid))
tabname = pftier[pftier_idx] + 's'
svfile = './' + tabname + '.xlsx'

df_GM = pd.DataFrame(columns= ['id','username','displayname','country','category'])

kgteam = 0
luid = len(uid)
for ii in range(0,luid):
    id = uid[ii]
    udf = users[users["Id"]==id]
    pt = udf.PerformanceTier.iloc[0]
    un = udf.UserName.iloc[0]
    dn = udf.DisplayName.iloc[0]
    cn = udf.Country.iloc[0]
    if pt == 5:
        kgteam += 1
    pd_tmp = gms_ach[gms_ach["UserId"]==id]
    df_GM.loc[ii] = [id, un, dn, cn, pd_tmp.AchievementType.iloc[0:len(pd_tmp)].tolist()]
    
print(kgteam)
print(df_GM)
print(df_GM.shape)

df_GM.to_excel(svfile, sheet_name=tabname,\
                columns = ['id','username','displayname','country','category'])

beep() # modified

3164
9
            id          username                       displayname  \
0          421           dremovd                    Dmitry Dryomov   
1          887            hugmee                            Hug Mi   
2         1300       brianelwell                      Brian Elwell   
3         1356        compassucl                     Lewis Griffin   
4         1996          mikhailz                           Mikhail   
...        ...               ...                               ...   
3159  24381106     olaflundstrom  Gustav Olaf Laitinen-Fredriksson   
3160  24573387        ayberkural                       Ayberk URAL   
3161  24939415         fajobgiua                          Chesthaa   
3162  25323600       jeeelsheikh                        Jana Hatem   
3163  25787455  analyticaobscura                            OzanM.   

             country        category  
0              Spain  [Competitions]  
1          Australia  [Competitions]  
2      United States  [Competitions

In [ ]:
def fetch_profile(luid,stind,df_GM):
    capabilities = {
        "browserName": "MicrosoftEdge",
        "ms:edgeOptions": {
            "args": ["--headless"]
        }
    }
    for ii in range(stind,luid):
        print(ii+1,'/',luid)
        username = df_GM.loc[ii,'username']
        displayname = df_GM.loc[ii,'displayname']
        print(displayname)

        profile_url = f'{KAGGLE_BASE_URL}{username}'
        df_GM.loc[ii,'ProfileUrl'] = profile_url
        print(profile_url)

        #fname = cachedir + "/" + f"{ii:04}" + "_" + username + ".txt"
        fname = cachedir + "/" + username + ".txt"
        if os.path.isfile(fname):
            with open(fname, encoding='utf-8') as f:
                bs = BeautifulSoup(f.read(), 'html.parser')
        else:
            driver = webdriver.Edge(executable_path='./msedgedriver', capabilities=capabilities) # locate appropriate webdriver in the executable-file directory
            driver.get(profile_url)
            time.sleep(sltime)
            bs = BeautifulSoup(driver.page_source.encode('utf-8'), 'html.parser')
            with open(fname, mode='w') as f:
                f.write(str(bs.prettify()))

        tmpstl = bs.find("div", {"class":"sc-iihIvb bUwKTg"}) # note that the class will be changed with a few days
        if not tmpstl:
            continue

        tmp = tmpstl['style']
        df_GM.loc[ii,'ImageUrl'] = cssutils.parseStyle(tmp)['background-image']
        
        tmplst = bs.find("ul",{"class":"sc-eFKCff gwCSOQ"}) # note that the class will be changed with a few days
        lis = tmplst.findAll("li")
        for ll in lis:
            tmpspan = ll.find("span").get_text().strip() #work, pin_drop, ... 
            tmpp = ll.find("p").get_text().strip()
            if re.search("work",tmpspan):
                tmplst = re.split(' at ',tmpp)
                if len(tmplst) == 0:
                    df_GM.loc[ii, 'Occupation'] = tmpp
                    print(tmpp)
                if len(tmplst) == 1:
                    print(tmpp)
                elif len(tmplst) == 2:
                    df_GM.loc[ii, 'Occupation'] = tmplst[0]
                    df_GM.loc[ii, 'Organization'] = tmplst[1]
                else:
                    print(tmpp)
            elif re.search("pin_drop",tmpspan):
                tmplst = re.split(', ',tmpp)
                if len(tmplst) == 0:
                    df_GM.loc[ii, 'Country'] = tmpp
                elif len(tmplst) == 1:
                    df_GM.loc[ii, 'Country'] = tmplst[0]
                    print(tmpp)
                elif len(tmplst) == 2:
                    df_GM.loc[ii, 'City'] = tmplst[0]
                    df_GM.loc[ii, 'Country'] = tmplst[1]
                elif len(tmplst) == 3:
                    df_GM.loc[ii, 'Region'] = tmplst[0]
                    df_GM.loc[ii, 'City'] = tmplst[1]
                    df_GM.loc[ii, 'Country'] = tmplst[2]
                else:
                    print(tmpp)

        tmplst = bs.find("div",{"class":"MuiTabs-flexContainer css-k008qs"})
        alst = tmplst.findAll("a")
        for aa in alst:
            tmp = aa.find("span").get_text().strip()
            if re.search("Competitions",tmp):
                df_GM.loc[ii, 'NumCompetitions'] = re.search("Competitions (\d*)",tmp)[0]
            elif re.search("Datasets",tmp):
                df_GM.loc[ii, 'NumDatasets'] = re.search("Datasets (\d*)",tmp)[0]
            elif re.search("Code",tmp):
                df_GM.loc[ii, 'NumCodes'] = re.search("Code (\d*)",tmp)[0]
            elif re.search("Discussion",tmp):
                df_GM.loc[ii, 'NumDiscussions'] = re.search("Discussion (\d*)",tmp)[0]
            elif re.search("Followers",tmp):
                df_GM.loc[ii, 'NumFollowers'] = re.search("Followers (\d*)",tmp)[0]
            elif re.search("Following",tmp):
                df_GM.loc[ii, 'NumFollowing'] = re.search("Following (\d*)",tmp)[0]


In [98]:
pklfl = './' + tabname + '.pkl'
svfile = '.' + tabname + '.xlsx'
cachedir = './.cache/' + tabname
if not os.path.exists(cachedir):
    os.makedirs(cachedir)
sltime = 5



fetch_profile(luid,0,df_GM)
df_GM.to_excel(svfile, sheet_name='GrandMasters',\
                columns = ['id','username','displayname',
                           'ProfileUrl',
                           'country','category',
                           'Occupation', 'Organization'])
beep() # modified



./.cache/Masters
1 / 3164
Dmitry Dryomov
https://kaggle.com/dremovd
2 / 3164
Hug Mi
https://kaggle.com/hugmee
3 / 3164
Brian Elwell
https://kaggle.com/brianelwell
['United States']
4 / 3164
Lewis Griffin
https://kaggle.com/compassucl
5 / 3164
Mikhail
https://kaggle.com/mikhailz
6 / 3164
Yannis Sismanis
https://kaggle.com/outis1
7 / 3164
Lee Baker
https://kaggle.com/leebaker
8 / 3164
Bluefool
https://kaggle.com/domcastro
['United Kingdom']
9 / 3164
Julien Meynet
https://kaggle.com/jmeynet
10 / 3164
Quan Sun
https://kaggle.com/quansun
['New Zealand']
11 / 3164
Adam
https://kaggle.com/adamhusein
12 / 3164
Jure Zbontar
https://kaggle.com/jzbontar
13 / 3164
Dirk N
https://kaggle.com/dirknbr
Google
14 / 3164
Chris Raimondi
https://kaggle.com/chrisraimondi
15 / 3164
Diogo R. Ferreira
https://kaggle.com/diogoff
16 / 3164
Uri Blass
https://kaggle.com/uriblass
17 / 3164
John Dunavent
https://kaggle.com/johndun
18 / 3164
Alec Stephenson
https://kaggle.com/alecstephenson
19 / 3164
Marcin Pionnier


AttributeError: 'str' object has no attribute 'get_text'